In [1]:
!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
!unzip "maestro-v3.0.0-midi.zip"
!rm "maestro-v3.0.0-midi.zip"

dataset_path = "/kaggle/working/maestro-v3.0.0"

--2024-11-28 14:10:23--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.207, 142.250.99.207, 173.194.202.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58416533 (56M) [application/octet-stream]
Saving to: 'maestro-v3.0.0-midi.zip'

maestro-v3.0.0-midi 100%[===================>]  55.71M   118MB/s    in 0.5s    

2024-11-28 14:10:24 (118 MB/s) - 'maestro-v3.0.0-midi.zip' saved [58416533/58416533]

Archive:  maestro-v3.0.0-midi.zip
  inflating: maestro-v3.0.0/2004/MIDI-Unprocessed_XP_08_R1_2004_01-02_ORIG_MID--AUDIO_08_R1_2004_01_Track01_wav.midi  
  inflating: maestro-v3.0.0/2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav.midi  
  inflating: maestro-v3.0.0/2004/MIDI-Unprocessed_XP_14_R1_2004_01-03_ORIG_MID--AUDIO_14_R1_2004_01_T

In [2]:
!pip install pretty_midi
!pip install miditok
!pip install keras_nlp
!pip install pretty_midi midi-clip
!pip install numba 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.4 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592292 sha256=12814e0c07ec5b01ebdd384477121ba46510f8b5c215e7ca7d4d3c03c294a68d
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.3 MB/s eta 0:00:0000:01


In [3]:
#@title Installing libraries to hear a MIDI

!apt-get update -qq && apt-get install -y fluidsynth


# PICONICA

!gdown 1uk51T9Gvo1n2JRl3_CHCg2FVGWiNI4qJ



# Utility library

!wget https://raw.githubusercontent.com/roostico/NesGen/refs/heads/main/utility.py

!wget https://raw.githubusercontent.com/roostico/NesGen/refs/heads/main/transformer.py



from utility import *

W: https://packages.cloud.google.com/apt/dists/gcsfuse-focal/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: https://packages.cloud.google.com/apt/dists/google-fast-socket/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libdouble-conversion3 libegl-mesa0 libegl1 libevdev2
  libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10 libinstpatch-1.0-2
  libmd4c0 libmtdev1 libpcre2-16-0 libqt5core5a libqt5dbus5 libqt5gui5
  libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common
  libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0
  libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0
  qsynth 

## Import libraries

In [4]:
import os

import random

import shutil

from tqdm import tqdm

from pathlib import Path

import pretty_midi

import numpy as np

from miditok import REMI, TokenizerConfig

from utility import playMidi, show_midi_info

import json

import keras_nlp.layers as nlp_layers

from tensorflow import keras

import tensorflow as tf

from miditok.utils import split_files_for_training

from miditok.data_augmentation import augment_dataset

from random import shuffle

In [5]:
skip = True

### Utility functions

In [6]:
def play_tokens(tokens: np.ndarray, tokenizer: REMI, delete_after: bool = True, show_info: bool = False):

  """

  Plays the given tokens, decoded using the given tokenizer

  """

  dumped_midi = "decoded.mid"

  tokenizer.decode([tokens]).dump_midi(dumped_midi)

  to_play = playMidi(dumped_midi)

  if show_info:

    show_midi_info(dumped_midi)

  if delete_after:

    os.remove(dumped_midi)

  return to_play





def random_filtered(collection: list, predicate):

  """

  Returns a random element from a collection that satisfies the given predicate.

  If no element satisfies the filter, returns None.

  """

  for elem in random.sample(collection, len(collection)):

    if predicate(elem):

      return elem



def lasts_less_than(midi_path: str, time_seconds: int) -> bool:

  """

  Returns true if the last note of the MIDI file is less than the given time in seconds.

  """

  return pretty_midi.PrettyMIDI(midi_path).get_end_time() <= time_seconds

## Move files and rename them

In [7]:
from pathlib import Path

import os

import shutil

import random



# Paths to the files of the dataset

midi_paths = list(Path(dataset_path).resolve().glob("**/*.mid")) + list(Path(dataset_path).resolve().glob("**/*.midi"))

midis_dir = "midis"
os.makedirs(midis_dir, exist_ok=True)


for i, midi_path in enumerate(midi_paths):

  new_midi_path = os.path.join(midis_dir, f"{i}.midi")

  shutil.move(str(midi_path), new_midi_path)



midis = list(Path("/kaggle/working/midis").resolve().glob("**/*.mid")) + list(Path("/kaggle/working/midis").resolve().glob("**/*.midi"))



def sample():

  return str(random.choice(midis))

## Select a sample of these files

### Define a filtering function

In [8]:
def is_valid(file: str) -> bool:

  """Checks if a MIDI file is valid. If any of its instruments has no name,

  it is invalid.



  Args:

      file: The path to the MIDI file.



  Returns:

      True if the MIDI file is valid, False otherwise.

  """

  try:

    midi = pretty_midi.PrettyMIDI(file)

    if any([len(instrument.name) == 0 for instrument in midi.instruments]):

      return False

    return True

  except Exception as e:

    print(e)

    return False

### Move files

In [9]:
def copy_random_files(source_dir: str, dest_dir: str, num_files: int, is_file_valid) -> list:

  """Copies a specified number of random files from a source directory to a destination directory.



  Args:

      source_dir: The path to the source directory.

      dest_dir: The path to the destination directory.

      num_files: The number of files to move.

  """

  if not os.path.exists(source_dir):

    print(f"Error: Source directory '{source_dir}' not found.")

    return



  if os.path.exists(dest_dir):

    shutil.rmtree(dest_dir)



  os.makedirs(dest_dir, exist_ok=True)

  files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]



  if len(files) < num_files:

    print(f"Warning: Only {len(files)} files found in '{source_dir}'. Moving all of them.")

    num_files = len(files)



  files_to_move = []

  i = 0

  with tqdm(total=num_files, position=0, leave=True) as pbar:

    while i < num_files:

      random_file = random.choice(files)

      if is_file_valid(os.path.join(source_dir, random_file)):

        files_to_move.append(random_file)

        i = i + 1

        pbar.update()

      else:

        continue



  result = []

  for file in files_to_move:

    source_path = os.path.join(source_dir, file)

    dest_path = os.path.join(dest_dir, file)

    shutil.copy(source_path, dest_path)

    result.append(dest_path)

  return result



source_directory = "midis"

destination_directory = "selected"

number_of_files_to_move = 1000



sample = copy_random_files(source_directory, destination_directory, number_of_files_to_move, lambda x: True)

assert len(sample) == number_of_files_to_move

100%|██████████| 1000/1000 [00:00<00:00, 447583.40it/s]


### Listen one random MIDI from sample

In [10]:
skip = False

In [11]:
if not skip:

  def valid_example(file: str) -> float:

    """

    Returns true if the midi file lasts less that 120 seconds and has more than 1 instrument

    """

    return lasts_less_than(file, 280) and len(pretty_midi.PrettyMIDI(file).instruments) > 1



  example = random_filtered(sample, lambda x: True)



  print(f"Showing file {example}")

  show_midi_info(example)

  playMidi(example)

Showing file selected/618.midi
Instruments:  ['6/29 1 ~ 3  e-Comp']
MIDI duration: 437.40 seconds


fluidsynth: error: fluid_is_soundfont(): fopen() failed: 'File does not exist.'
Parameter '/content/guGS.sf2' not a SoundFont or MIDI file or error occurred identifying it.


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'audio.wav'..


## Pre-processing

## Tokenizer




In [12]:
BEAT_RES = {(0, 1): 12, (1, 2): 4, (2, 4): 2, (4, 8): 1}

TOKENIZER_PARAMS = {

    "pitch_range": (21, 109),

    "beat_res": BEAT_RES,

    "num_velocities": 24,

    "special_tokens": ["PAD", "BOS", "EOS"],

    "use_chords": True,

    "use_rests": True,

    "use_tempos": True,

    "use_time_signatures": True,

    "use_programs": False,  # no multitrack here

    "num_tempos": 32,

    "tempo_range": (50, 200),  # (min_tempo, max_tempo)

}

config = TokenizerConfig(**TOKENIZER_PARAMS)

tokenizer = REMI(config)

### (Optional): train the tokenizer

In [13]:
vocab_size=10000
tokenizer.train(vocab_size=vocab_size, files_paths=sample)

In [14]:
processed = [Path(f"/kaggle/working/{s}") for s in sample]
print(len(processed))
print(processed[0])

1000
/kaggle/working/selected/1213.midi


In [15]:
total_num_files = len(processed)

num_files_valid = round(total_num_files * 0.15)

num_files_test = round(total_num_files * 0.15)

shuffle(processed)

midi_paths_valid = processed[:num_files_valid]

midi_paths_test = processed[num_files_valid:num_files_valid + num_files_test]

midi_paths_train = processed[num_files_valid + num_files_test:]



# Chunk MIDIs and perform data augmentation on each subset independently

for files_paths, subset_name in (

    (midi_paths_train, "train"), (midi_paths_valid, "valid"), (midi_paths_test, "test")

):
    print(files_paths[0])

    # Split the MIDIs into chunks of sizes approximately about 1024 tokens

    subset_chunks_dir = Path(f"Maestro_{subset_name}")

    split_files_for_training(

        files_paths=files_paths,

        tokenizer=tokenizer,

        save_dir=subset_chunks_dir,

        max_seq_len=1024,

        num_overlap_bars=2,

    )



    # Perform data augmentation

    augment_dataset(

        subset_chunks_dir,

        pitch_offsets=[-12, 12],

        velocity_offsets=[-4, 4],

        duration_offsets=[-0.5, 0.5],

    )
midi_paths_train = list(Path("Maestro_train").glob("**/*.mid")) + list(Path("Maestro_train").glob("**/*.midi"))

midi_paths_valid = list(Path("Maestro_valid").glob("**/*.mid")) + list(Path("Maestro_valid").glob("**/*.midi"))

midi_paths_test = list(Path("Maestro_test").glob("**/*.mid")) + list(Path("Maestro_test").glob("**/*.midi"))


/kaggle/working/selected/713.midi


Performing data augmentation: 100%|██████████| 5540/5540 [00:15<00:00, 363.14it/s]


/kaggle/working/selected/1165.midi


Performing data augmentation: 100%|██████████| 1442/1442 [00:03<00:00, 367.11it/s]


/kaggle/working/selected/1135.midi


Performing data augmentation: 100%|██████████| 1536/1536 [00:04<00:00, 363.72it/s]


### Tokenizer the dataset

In [16]:
def midi_valid(midi) -> bool:

    if any(ts.numerator != 4 for ts in midi.time_signature_changes):

        return False  # time signature different from 4/*, 4 beats per bar

    return True



if os.path.exists("tokenized"):

  shutil.rmtree("tokenized")


for dir in ("train", "valid", "test"):
    tokenizer.tokenize_dataset(        
    
        Path(f"/kaggle/working/Maestro_{dir}"),
    
        Path(f"/kaggle/working/tokenized_{dir}"),
    
        midi_valid,
    
    )

Tokenizing music files (working/tokenized_test): 100%|██████████| 9199/9199 [03:35<00:00, 42.71it/s]


### Utility function to read a JSON tokenized file

In [17]:
def read_json(path: str) -> dict:

  with open(path, "r") as f:

    return json.load(f)

## Read the tokenized version of files from the JSON

In [18]:
def read_json_files(json_file_paths):

    """Reads a list of JSON files and returns a list of objects.



    Args:

        json_file_paths: A list of file paths to JSON files.



    Returns:

        A list of objects, where each object represents the data from a JSON file.

        Returns an empty list if any error occurs during file processing.

    """

    objects = []

    for file_path in tqdm(json_file_paths):

        try:

            objects.append(read_json(file_path))

        except FileNotFoundError:

            print(f"Error: File not found - {file_path}")

            return [] # Return empty list on error

        except json.JSONDecodeError:

            print(f"Error decoding JSON in file: {file_path}")

            return [] # Return empty list on error

    return objects



# Example usage (assuming 'tokenized' directory contains JSON files):

tokenized_train = list(Path("tokenized_train").resolve().glob("**/*.json"))
data_objects_train = read_json_files(tokenized_train)

tokenized_valid = list(Path("tokenized_valid").resolve().glob("**/*.json"))
data_objects_valid = read_json_files(tokenized_valid)

tokenized_test = list(Path("tokenized_test").resolve().glob("**/*.json"))
data_objects_test = read_json_files(tokenized_test)


if data_objects_train:

    print(f"\nSuccessfully read {len(data_objects_train)} JSON files.")

    # Now you can work with the 'data_objects' list

    # For example, print the first object:

    # print(data_objects[0])

else:

    print("Error reading JSON files.")

100%|██████████| 9199/9199 [00:01<00:00, 6033.53it/s]


Successfully read 33269 JSON files.


## Create the list of tokenized songs, taking the IDs of each one

In [19]:
encoded_train = [np.array(song["ids"][0]) for song in data_objects_train]
encoded_valid = [np.array(song["ids"][0]) for song in data_objects_valid]
encoded_test = [np.array(song["ids"][0]) for song in data_objects_test]

End of pre-processing, proceding with data and model preparation with Tensorflow



---




# Tensorflow data and model setup

## Creating a Tensorflow dataset with all IDs

In [20]:
import tensorflow as tf

import numpy as np



all_ids_train = np.concatenate(encoded_train)
all_ids_valid = np.concatenate(encoded_valid)
all_ids_test = np.concatenate(encoded_test)

ids_dataset_train = tf.data.Dataset.from_tensor_slices(all_ids_train)
ids_dataset_valid = tf.data.Dataset.from_tensor_slices(all_ids_valid)
ids_dataset_test = tf.data.Dataset.from_tensor_slices(all_ids_test)

### Convert into sequences

In [21]:
seq_length = 1024 #@param {type: 'slider', max: 500, min: 50, step: 50}

sequences_train = ids_dataset_train.batch(seq_length+1, drop_remainder=True)
sequences_valid = ids_dataset_valid.batch(seq_length+1, drop_remainder=True)
sequences_test = ids_dataset_test.batch(seq_length+1, drop_remainder=True)

## Preparing labels

In [22]:
def split_input_target(sequence):
    # Convert to float32
    input_seq = tf.cast(sequence[:-1], tf.int32)
    target_seq = tf.cast(sequence[1:], tf.int32)
    return input_seq, target_seq



train_ds = sequences_train.map(split_input_target)
valid_ds = sequences_valid.map(split_input_target)
test_ds = sequences_test.map(split_input_target)

### Creating training batches

In [23]:
# Batch size

BATCH_SIZE = 16 #@param {type: 'slider', max: 256, min: 32, step: 32}



# Buffer size to shuffle the dataset

# (TF data is designed to work with possibly infinite sequences,

# so it doesn't attempt to shuffle the entire sequence in memory. Instead,

# it maintains a buffer in which it shuffles elements).

BUFFER_SIZE = 10000



train_ds = (
    train_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
valid_ds = (
    valid_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
test_ds = (
    test_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))



# Building the model

In [24]:
!pip install transformers
!pip install accelerate


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## GAN

In [26]:
def build_gan(input_noise_dim, target_count, hidden_activation='relu', 
              generator_output_activation='sigmoid', 
              unit_count_per_rnn_layer=[128, 128]):

    # Generatore
    generator = keras.Sequential(name='generator')
    generator.add(layers.Input(shape=(None, input_noise_dim), name='generator_input'))

    for i in range(len(unit_count_per_rnn_layer)):
        generator.add(layers.SimpleRNN(
            unit_count_per_rnn_layer[i], 
            activation=hidden_activation, 
            return_sequences=(i < len(unit_count_per_rnn_layer) - 1)
        ))
    
    generator.add(layers.Dense(target_count, activation=generator_output_activation, name='generator_output'))

    # Discriminatore
    discriminator = keras.Sequential(name='discriminator')
    discriminator.add(layers.Input(shape=(target_count,), name='discriminator_input'))

    for neuron_count in reversed(unit_count_per_rnn_layer):
        discriminator.add(layers.Dense(neuron_count, activation=hidden_activation))
    
    discriminator.add(layers.Dense(1, activation='sigmoid', name='discriminator_output'))

    # GAN
    gan_input = layers.Input(shape=(None, input_noise_dim), name='gan_input')
    gan_output = discriminator(generator(gan_input))
    gan = keras.Model(gan_input, gan_output, name='gan')

    return gan, generator, discriminator

In [27]:
noise_dim=100

gan,gan_generator,gan_discriminator=build_gan(noise_dim, seq_length)

In [28]:
gan.summary()

Model: "gan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gan_input (InputLayer)          │ (None, None, 100)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ generator (Sequential)          │ (None, 1024)           │       194,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ discriminator (Sequential)      │ (None, 1)              │       147,841 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 342,145 (1.31 MB)

 Trainable params: 342,145 (1.31 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
gan_discriminator.compile(loss='binary_crossentropy', optimizer='sgd')

gan_discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer='sgd')

### GAN utils

In [86]:
def chunks(list, n):
    for i in range(0, len(list), n):
        yield list[i:i + n]

def get_random_batch_indices(data_count,batch_size):
    list_indices=list(range(0,data_count))
    random.shuffle(list_indices)
    return list(chunks(list_indices, batch_size))

def get_gan_real_batch(dataset_x,batch_indices,label):
    batch_x = [x for x, _ in dataset_x.take(1)]
    batch_y=np.full(len(batch_indices),label)

    return batch_x,batch_y

In [91]:
def get_gan_random_input(batch_size,noise_dim,*_):
  return np.random.normal(0, 1, size=(batch_size, noise_dim))

def get_gan_fake_batch(generator,batch_size,generator_input):
  batch_x=generator.predict(tf.expand_dims(generator_input, axis=1),verbose=0)
  batch_y=np.zeros(batch_size)

  return batch_x,batch_y

In [88]:
def concatenate_gan_batches(real_batch_x,fake_batch_x):
    fake_tensor = tf.convert_to_tensor(fake_batch_x, dtype=tf.int32)
    return tf.concat([real_batch_x[0], fake_tensor], axis=0)

## GAN training

In [83]:
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [94]:
import time
from tqdm import tqdm

def train_gan(gan,generator,discriminator,train_x,train_data_count,input_noise_dim,epoch_count,batch_size,
              get_random_input_func,get_real_batch_func,get_fake_batch_func,concatenate_batches_func,vocab_size,condition_count=-1,
              use_one_sided_labels=False):
    iteration_count = int(train_data_count / batch_size)

    print('Epochs: ', epoch_count)
    print('Batch size: ', batch_size)
    print('Iterations: ', iteration_count)
    print('')

    d_epoch_losses=[]
    g_epoch_losses=[]
    for e in tqdm(range(1, epoch_count+1)):
        start_time = time.time()
        avg_d_loss=0
        avg_g_loss=0

        # Training indices are shuffled and grouped into batches
        batch_indices=get_random_batch_indices(train_data_count,batch_size)

        for i in tqdm(range(iteration_count)):
            current_batch_size=len(batch_indices[i])

            # 1. create a batch with real songs from the training set
            real_batch_x,real_batch_y=get_real_batch_func(train_x,batch_indices[i],0.9 if use_one_sided_labels else 1)

            print(f'real_x: {real_batch_x}\n\nreal_y: {real_batch_y}')
            
            # 2. create noise vectors for the generator and generate songs from the noise
            generator_input=get_random_input_func(current_batch_size, input_noise_dim,condition_count)

            print(f'gen_input: {generator_input}')
            
            fake_batch_x,fake_batch_y=get_fake_batch_func(generator,current_batch_size,generator_input)

            print(f'fake_x: {fake_batch_x}\n\nfake_y: {fake_batch_y}')

            print(f'fake_x_len: {len(fake_batch_x)}\n\nfake_x_sub_len: {len(fake_batch_x[0])}')
            
            # batch_x = [[int(el*multiplier) for el in sub_list] for sub_list in batch_x]

            # 3. concatenate real and fake batches into a single batch
            discriminator_batch_x = concatenate_batches_func(real_batch_x, fake_batch_x)
            discriminator_batch_y= np.concatenate((real_batch_y, fake_batch_y))

            # 4. train discriminator
            d_loss = discriminator.train_on_batch(discriminator_batch_x, discriminator_batch_y)

            # 5. create noise vectors for the generator
            gan_batch_x = get_random_input_func(current_batch_size, input_noise_dim,condition_count)
            gan_batch_y = np.ones(current_batch_size)    #Flipped labels

            gan_batch_extended = tf.expand_dims(gan_batch_x, axis=1)

            # 6. train generator
            g_loss = gan.train_on_batch(gan_batch_extended, gan_batch_y, return_dict=True)

            # 7. avg losses
            avg_d_loss+=d_loss*current_batch_size
            avg_g_loss+=g_loss['loss']*current_batch_size

        avg_d_loss/=train_data_count
        avg_g_loss/=train_data_count

        d_epoch_losses.append(avg_d_loss)
        g_epoch_losses.append(avg_g_loss)

        end_time = time.time()

        print('Epoch: {0} exec_time={1:.1f}s d_loss={2:.3f} g_loss={3:.3f}'.format(e,end_time - start_time,avg_d_loss,avg_g_loss))

    return d_epoch_losses,g_epoch_losses

In [95]:
epoch_count=10
input_noise_dim=100

d_epoch_losses,g_epoch_losses=train_gan(gan,
                                        gan_generator,
                                        gan_discriminator,
                                        train_ds,
                                        len(train_ds),
                                        input_noise_dim,
                                        epoch_count,
                                        BATCH_SIZE,
                                        get_gan_random_input,
                                        get_gan_real_batch,
                                        get_gan_fake_batch,
                                        concatenate_gan_batches,
                                        vocab_size,)

Epochs:  10
Batch size:  16
Iterations:  130



  0%|          | 0/130 [00:00<?, ?it/s]

real_x: [<tf.Tensor: shape=(16, 1024), dtype=int32, numpy=
array([[ 474,   47,  406, ..., 2227,  291, 6545],
       [1202, 3759,   58, ...,  187, 1431, 2532],
       [ 372, 4308,  328, ...,  342, 2387,  369],
       ...,
       [ 151, 9083,  153, ..., 3715,  388, 3359],
       [ 928, 3615,  303, ...,  569,  187,  563],
       [ 162,   81,  684, ...,  515,  145, 1766]], dtype=int32)>]

real_y: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
gen_input: [[-0.06103808 -0.38417667  0.7107681  ...  0.64120447  1.13368016
  -0.80780529]
 [-0.90713392  0.27863126  1.4686239  ... -0.95117584 -0.03058658
  -0.29844508]
 [-0.93939934  1.45375056 -0.82117238 ... -0.47776635  1.23985985
  -0.40824663]
 ...
 [ 0.92402279 -0.31427302  0.66410185 ...  1.1629415  -0.15844192
  -0.30660891]
 [-2.16911472 -0.37468092 -1.0513803  ...  0.89352602 -0.79851602
   1.50705076]
 [-0.64599933  0.21267409 -0.15458295 ...  1.02568875 -1.28146393
   1.30078407]]
fake_x: [[0.3130173  0.43047222 0.70774955 ... 0.6257379  0.4679482


  1%|          | 1/130 [00:10<22:18, 10.38s/it]

real_x: [<tf.Tensor: shape=(16, 1024), dtype=int32, numpy=
array([[ 668, 8170, 5172, ...,  363, 2950,  364],
       [7036,  372,   43, ...,  265, 3980,  201],
       [ 677, 1734,  382, ..., 1436,  631, 2153],
       ...,
       [1141, 3275,  440, ...,  342, 7513, 1744],
       [2684, 5285,  430, ...,   47, 4293,  594],
       [ 749,  170,  885, ...,  319, 3504,  429]], dtype=int32)>]

real_y: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
gen_input: [[-0.25695525  1.1210268   1.36398755 ...  0.69179366  0.89070404
  -0.99048231]
 [ 0.65772327  0.74959991  1.54804533 ...  0.82789866 -0.578893
   0.07214563]
 [ 0.02352377 -0.57290942  0.92118499 ...  0.50982015  0.83196004
   1.22027292]
 ...
 [-0.99545976 -2.35090038 -0.42811814 ...  1.24270102  0.37162099
   0.78509589]
 [-0.30047912  0.77181651  0.35711716 ... -2.21350012  0.85970837
   0.86173509]
 [ 1.18162345  0.90042214 -0.090819   ...  0.30572961 -0.74321791
  -0.48621054]]
fake_x: [[0.32973707 0.33301422 0.6991924  ... 0.74552846 0.48696804 

...


  8%|▊         | 11/130 [01:53<20:27, 10.31s/it]

real_x: [<tf.Tensor: shape=(16, 1024), dtype=int32, numpy=
array([[7076, 4382,  342, ...,   46,  393, 5734],
       [4476,  374,  158, ..., 3089, 2727,   12],
       [ 169,  971, 3027, ..., 4244,  146, 1066],
       ...,
       [ 152,  811,  154, ...,   77,  545,   41],
       [ 186,  717,  541, ..., 8196,  350, 9718],
       [ 739,   60, 4612, ...,  169, 1482,  170]], dtype=int32)>]

real_y: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
gen_input: [[-0.58064627 -0.13541804  0.19180721 ...  0.79474566 -0.9076117
  -2.06793705]
 [ 0.91032724 -1.38613255  1.52202221 ...  0.60616486 -0.87419016
  -0.87738037]
 [ 0.20466725 -0.52224971  1.37480965 ... -1.45294593 -0.66344347
   0.0659292 ]
 ...
 [-0.78212893  0.44377699  0.62453728 ...  0.81036226 -0.64933314
   0.16986374]
 [-0.89173316  0.01250632  0.76367287 ... -1.5017391  -0.46157649
  -0.68562435]
 [ 0.45763525  2.15150097 -0.3588051  ...  1.35826446  1.62517948
  -1.19853508]]
fake_x: [[0.34339634 0.39350984 0.6530793  ... 0.66778314 0.52806526


  0%|          | 0/10 [02:13<?, ?it/s]

real_x: [<tf.Tensor: shape=(16, 1024), dtype=int32, numpy=
array([[9932, 1620,   46, ...,  765, 2961, 1021],
       [  77,  462, 7696, ..., 4144,  552, 9085],
       [3885, 5260, 2731, ...,  366, 2825,  278],
       ...,
       [ 447,   35,  534, ...,   49,  449,   56],
       [ 308,   28,  357, ..., 7462, 8915, 2845],
       [ 789,  173,  573, ...,  704, 4687, 1683]], dtype=int32)>]

real_y: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
gen_input: [[-1.03288726 -1.27961133 -0.61744922 ...  2.16996317  0.20816069
   0.80055636]
 [-0.48966381  1.07800097  0.05064364 ... -0.49602504  0.38711075
  -0.845429  ]
 [ 1.16742732 -0.06441918  0.22886149 ... -0.84538775 -0.28648596
   0.07637707]
 ...
 [ 0.22647093  0.9432837   0.9179825  ...  0.54749178  0.77972484
   0.74206243]
 [-0.93767798  0.28647175 -0.10723058 ... -0.57437174 -1.84405844
  -1.11855353]
 [ 0.57758134  1.13598167 -1.36045065 ...  1.90201848 -2.16797333
  -0.12808104]]


KeyboardInterrupt: 

#### Best practices in training GAN

In [ ]:
# Normalize input

train_x_flatten = (train_x_flatten*2)-1
val_x_flatten = (val_x_flatten*2)-1
test_x_flatten = (test_x_flatten*2)-1

generator_output_activation='tanh'

hidden_activation=layers.LeakyReLU(alpha=0.2)

use_one_sided_labels=True

optimizer = keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

In [ ]:
epoch_count=30
batch_size=100

gan,gan_generator,gan_discriminator=build_gan(input_noise_dim,
                                              [256,512,1024],
                                              train_x_flatten.shape[1],
                                              hidden_activation,
                                              generator_output_activation)

gan_discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

gan_discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

d_epoch_losses,g_epoch_losses=train_gan(gan,
                                        gan_generator,
                                        gan_discriminator,
                                        val_x_flatten,
                                        val_x_flatten.shape[0],
                                        input_noise_dim,
                                        epoch_count,
                                        batch_size,
                                        get_gan_random_input,
                                        get_gan_real_batch,
                                        get_gan_fake_batch,
                                        concatenate_gan_batches,
                                        use_one_sided_labels=use_one_sided_labels,
                                        plt_frq=1,
                                        plt_example_count=15)

### Check if model is doing what expected

In [29]:
for input_example_batch, target_example_batch in train_ds.take(1):

  example_batch_predictions = model(input_example_batch)
  logits = example_batch_predictions.logits
  print(logits.shape, "# (batch_size, sequence_length, vocab_size)")



# Check shapes
print("Prediction shape:", logits.shape)
print("Target shape:", target_example_batch.shape)

# Ensure reduction is feasible
predicted_classes = tf.argmax(logits, axis=-1)  # (batch_size, seq_length)
print("Reduced prediction shape:", predicted_classes.shape)

# Compare shapes after reduction
if predicted_classes.shape == target_example_batch.shape:
    print("Shapes are compatible for comparison.")
else:
    print("Shapes are NOT compatible for comparison.")

# Verify dtype compatibility
print("Prediction dtype:", logits.dtype)
print("Target dtype:", target_example_batch.dtype)

(16, 1024, 10000) # (batch_size, sequence_length, vocab_size)
Prediction shape: (16, 1024, 10000)
Target shape: (16, 1024)
Reduced prediction shape: (16, 1024)
Shapes are compatible for comparison.
Prediction dtype: <dtype: 'float32'>
Target dtype: <dtype: 'int32'>


In [31]:
# Directory where the checkpoints will be saved

checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

early_stopping = tf.keras.callbacks.EarlyStopping(

    monitor='val_loss',

    patience=3,

    restore_best_weights=True

)



checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(

    filepath=checkpoint_prefix,

    save_weights_only=True

)

In [ ]:
EPOCHS = 5



history = model.fit(train_ds,

                    epochs=EPOCHS,

                    validation_data=valid_ds,

                    )

Epoch 1/5
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1732292991.331854    1158 service.cc:145] XLA service 0x7ad37c002fd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732292991.331936    1158 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
W0000 00:00:1732292991.886217    1158 assert_op.cc:38] Ignoring Assert operator tf_mistral_for_causal_lm/model/assert_less/Assert/Assert
I0000 00:00:1732293003.802137    1158 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 280/2104 [==>...........................] - ETA: 21:25 - loss: 7.1729 - sparse_categorical_accuracy: 0.0118

In [ ]:
model.save_pretrained("/kaggle/working/nesgen-automodel.h5")

In [ ]:
!zip -r nesgen-automodel.zip /kaggle/working/nesgen-automodel.h5

## Generation

In [ ]:
def generate_one_step(model, seed):

  predictions = model(seed)

  sampled_indices = tf.random.categorical(predictions[0], num_samples=1)

  sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

  return sampled_indices



for seed_ids, _ in test_ds.take(1):

  seed = seed_ids

seed = seed[0]

In [ ]:
input_ids = tf.convert_to_tensor(seed)  # Assuming seed is a 1D tensor of token IDs
input_ids = tf.expand_dims(input_ids, 0)  # Add an extra dimension to represent batch size

# Generate continuation
outputs = model.generate(
    input_ids=input_ids,
    max_new_tokens=512,  # Maximum length of generated sequence
    num_return_sequences=1,  # Number of sequences to return
    do_sample=True,  # Use sampling (True) or greedy decoding (False)
    temperature=0.7  # Sampling temperature (lower is more conservative)
)

input_length = input_ids.shape[1]
generated_tokens = outputs[:, input_length:] # skip seed

# Decode and print the generated text
generated = tokenizer.decode([generated_tokens[0]])
generated.dump_midi("nesgen2.mid")

In [ ]:
midi = tokenizer.decode([seed])

midi.dump_midi("seed.mid")

In [ ]:
def generate(seed, model, length = 5):

  import time

  start = time.time()

  next_ids = seed

  result = [next_ids]

  n = 0

  for n in tqdm(range(length)):

    new_id = generate_one_step(model, tf.expand_dims(next_ids, axis=0))[-1]
    result.append([new_id])
    next_ids = np.append(next_ids, new_id)[1:]



  end = time.time()

  print('\nRun time:', end - start, "\n", '_'*80, "\n")

  result = np.concatenate(result[1:])

  print("Shape of result: ", result.shape)

  return result

In [ ]:
result = generate(seed, model, 1000)

In [ ]:
result

In [ ]:
midi = tokenizer.decode([result])

midi.dump_midi("test2.mid")

#playMidi("nesgen.mid")

---

## Try generating with un-trained model

In [ ]:
from utility import playMidi



# Use the model

for input_example_batch, target_example_batch in dataset.take(1):

  example_batch_predictions = model(input_example_batch)

  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")



# Sample indices from predictions

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()



# Decode

midi = tokenizer.decode([sampled_indices])

midi.dump_midi("boh5.mid")

playMidi("boh5.mid")

In [ ]:
# Calculate the loss

#loss = tf.keras.losses.sparse_categorical_crossentropy(target_example_batch, example_batch_predictions, from_logits=True)

# Reduce mean to get a single scalar loss value

#loss = tf.reduce_mean(loss)



#print("Loss:", loss.numpy())

## Taken from tensorflow tutorial:



A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
#print("Checking if it is near to vocabulary size")

#print(tf.exp(example_batch_mean_loss).numpy())

#print("Vocab size: ", encoding.vocab_size)

# Training

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [ ]:
# Directory where the checkpoints will be saved

checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

early_stopping = tf.keras.callbacks.EarlyStopping(

    monitor='val_loss',

    patience=3,

    restore_best_weights=True

)



checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(

    filepath=checkpoint_prefix,

    save_weights_only=True

)

In [ ]:
EPOCHS = 5



history = model.fit(train_ds,

                    epochs=EPOCHS,

                    validation_data=valid_ds,

                    callbacks=[checkpoint_callback, early_stopping]

                    )

# Generation

In [ ]:
class OneStep(tf.keras.Model):

  def __init__(self, model, decoding, encoding, vocab_size, temperature=1.0):

    super().__init__()

    self.temperature = temperature

    self.model = model

    self.decode = decoding

    self.encode = encoding



    # Taken from tensorflow tutorial: useful to skip ids



    #skip_ids = self.ids_from_chars(['[UNK]'])[:, None]

    #sparse_mask = tf.SparseTensor(

        # Put a -inf at each bad index.

    #    values=[-float('inf')]*len(skip_ids),

    #    indices=skip_ids,

        # Match the shape to the vocabulary

    #    dense_shape=[vocab_size])

    #self.prediction_mask = tf.sparse.to_dense(sparse_mask)



  @tf.function

  def generate_one_step(self, input_ids):

    input_ids_ = tf.expand_dims(input_ids, axis=0)



    # Run the model.

    # predicted_logits.shape is [batch, char, next_char_logits]

    predicted_logits = self.model(inputs=input_ids_)

    # Only use the last prediction.

    predicted_logits = predicted_logits[:, -1, :]

    predicted_logits = predicted_logits/self.temperature



    # Taken from tensorflow tutorial: apply prediction mask to prevent certain

    # ids from being generated

    #predicted_logits = predicted_logits + self.prediction_mask



    # Sample the output logits to generate token IDs.

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)

    predicted_ids = tf.squeeze(predicted_ids, axis=-1)



    # Return the ids and model state.



    return predicted_ids

In [ ]:
one_step_model = OneStep(model, tokenizer.decode, tokenizer.encode, tokenizer.vocab_size)

In [ ]:
midis = list(Path("/content/midis").glob("*.mid"))

In [ ]:
def get_random_file() -> str:

  return random.choice(midis)



seed_ids = np.array(tokenizer.encode(get_random_file())[0].ids)

seed_ids = seed_ids[:seq_length]

print(seed_ids)

In [ ]:
tf.config.run_functions_eagerly(True)

one_step_model.generate_one_step(seed_ids)

In [ ]:
import time



start = time.time()

next_ids = seed_ids

result = [next_ids]

n = 0

while n < 1000:

  next_ids = one_step_model.generate_one_step(next_ids)

  if next_ids == tokenizer.vocab["Bar_None"]:

    continue

  n = n + 1

  result.append(next_ids)



end = time.time()

print('\nRun time:', end - start, "\n", '_'*80, "\n")

result = np.concatenate(result[1:])

print("Shape of result: ", result.shape)

## Save the generator

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')

## Reload the generator

In [ ]:
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
def tokens_from_ids(ids, tokenizer):

  tokens = []

  for id in ids:

    for key, value in tokenizer.vocab.items():

      if value == id:

        tokens.append(key)

  return np.array(tokens)

In [ ]:
example = train_ds.take(1).as_numpy_iterator().next()[0]

In [ ]:
tokens_from_ids(example[0], tokenizer)

In [ ]:
tokens_from_ids(result, tokenizer)

## Hear the result

In [ ]:
print(tokens_from_ids(result, tokenizer))

In [ ]:
from utility import playMidi



midi = tokenizer.decode([result])

midi.dump_midi("result.mid")

playMidi("result.mid")